# Use PyEZ for Juniper ONBOX automation task

Junos 16.1R3 and later we can use PyEZ for on box automation, just like shell script used in older relase.

A python script can be used with event-option script, op script, and commit script

For python scripts to work we need to set script language as python:

#set system scripts language python
#commit

We would use jcs module to achieve different tasks 

To debug the python script on juniper router configure it as op script and run from junos cli


In [ ]:
# To check the python version

    root@napa% vi python_print_version.py 
    #!/usr/bin/env python

    import platform

    print(platform.sys.version)


Output:
    
    root@napa> op python_print_version.py 
    2.7.8 (default, Dec  5 2020, 00:22:45) 
    [GCC 4.2.1 (for JUNOS)]


In [ ]:
# print log into log message file using jcs.syslog function 

# jcs.syslog(priority, message, <message2>)

# https://www.juniper.net/documentation/en_US/junos/topics/reference/scripting/junos-script-automation-function-jcs-syslog.html

Example:

    % vi print_log_version.py 

    #!usr/bin/env python

    import platform

    import jcs

    print('This is a python op script')

    print('Printing python version in log messages')

    jcs.syslog('user.err','Print log to log messages')

    jcs.syslog('user.err',platform.sys.version)


Output:
    
    
    labroot@napa> op print_log_version.py 
    This is a python op script
    Printing python version in log messages
    
    
    labroot@napa> show log messages | last 20 
    Jan 20 12:06:17  napa mgd[16583]: UI_CMDLINE_READ_LINE: User 'labroot', command 'op print_log_version.py '
    Jan 20 12:06:18  napa cscript: Print log to log messages
    Jan 20 12:06:18  napa cscript: 2.7.8 (default, Dec  5 2020, 00:22:45)  [GCC 4.2.1 (for JUNOS)]
    


In [ ]:
# using argparse in op script 


Example:
    
    #!/usr/bin/env python

    import jcs 

    import argparse

    print('This is a python op script') 

    print('Op script arg parser')

    parser = argparse.ArgumentParser()

    parser.add_argument('-arg1') 

    parser.add_argument('-arg2')

    args = parser.parse_args()

    print('Taking two args from Junos cli: {} {}'.format(args.arg1,args.arg2))

    
Output:

    labroot@napa# run op python_argparse.py arg1 OSPF arg2 BGP 
    This is a python op script
    Op script arg parser
    Taking two args from Junos cli: OSPF BGP



# Arguments can be supplied via event-option confuiguration as well:

    set event-options policy bgp_data_collection events rpd_bgp_neighbor_state_changed
    set event-options policy bgp_data_collection within 120 trigger until
    set event-options policy bgp_data_collection within 120 trigger 5
    set event-options policy bgp_data_collection attributes-match rpd_bgp_neighbor_state_changed.new-state matches Idle
    set event-options policy bgp_data_collection attributes-match rpd_bgp_neighbor_state_changed.old-state matches Established
    set event-options policy bgp_data_collection then event-script jnpr_bgp_data_collection.py arguments peer "{$$.peer-name}"
    set event-options policy bgp_data_collection then event-script jnpr_bgp_data_collection.py arguments old_state "{$$.old-state}"
    set event-options policy bgp_data_collection then event-script jnpr_bgp_data_collection.py arguments new_state "{$$.new-state}"
    set event-options policy bgp_data_collection then event-script jnpr_bgp_data_collection.py arguments group_filter IRR-MESH-IPV
    set event-options event-script file jnpr_bgp_data_collection.py python-script-user <username>
    set event-options event-script optional
    set system scripts language python
    set protocols bgp log-updown
    
    https://css-git.juniper.net/blainew/MS_BGP_FLAP_COLLECT/blob/master/jnpr_bgp_data_collection.py
        

In [ ]:
# Get the shell output using PyEZ

Example:

    % vi python_run_command.py 

    #!/usr/bin/env python

    import subprocess

    print('Op script interact with cli command')
    print('show version\n')
    cmd='cli -c "show version"'
    output=subprocess.check_output(cmd,shell=True)
    print(output)

Output:
    
    labroot@napa> op python_run_command.py 
    Op script interact with cli command
    show version
    Hostname: napa
    Model: mx80
    Junos: 18.2X75-D61.10
    JUNOS Base OS boot [18.2X75-D61.10]
    JUNOS Base OS Software Suite [18.2X75-D61.10]
    JUNOS Crypto Software Suite [18.2X75-D61.10]
    JUNOS Packet Forwarding Engine Support (MX80) [18.2X75-D61.10]
    JUNOS Web Management [18.2X75-D61.10]
    JUNOS Online Documentation [18.2X75-D61.10]
    JUNOS SDN Software Suite [18.2X75-D61.10]
    JUNOS Services Application Level Gateways [18.2X75-D61.10]
    JUNOS Services COS [18.2X75-D61.10]
    JUNOS Services Jflow Container package [18.2X75-D61.10]
    JUNOS Services Stateful Firewall [18.2X75-D61.10]
    JUNOS Services NAT [18.2X75-D61.10]
    JUNOS Services RPM [18.2X75-D61.10]
    JUNOS Services Captive Portal and Content Delivery Container package [18.2X75-D61.10]
    JUNOS Macsec Software Suite [18.2X75-D61.10]
    JUNOS Services Crypto [18.2X75-D61.10]
    JUNOS Services IPSec [18.2X75-D61.10]
    JUNOS DP Crypto Software Software Suite [18.2X75-D61.10]
    JUNOS py-base-powerpc [18.2X75-D61.10]
    JUNOS py-extensions-powerpc [18.2X75-D61.10]
    JUNOS jsd [powerpc-18.2X75-D61.10-jet-1]
    JUNOS Kernel Software Suite [18.2X75-D61.10]
    JUNOS Routing Software Suite [18.2X75-D61.10]
    

Another Example on how to run shell command: 
    
    labroot@napa> file show /var/db/scripts/op/python_run_command.py    
 
    #!/usr/bin/env python

    import subprocess
    
    print('Op script interact with cli command')
    print('uname -a\n')
    cmd='uname -a'
    
    output=subprocess.check_output(cmd,shell=True)
    print(output)
    
    
    labroot@napa> op python_run_command.py 
    Op script interact with cli command
    uname -a
    JUNOS napa 18.2X75-D61.10 JUNOS 18.2X75-D61.10 #0: 2020-12-05 01:08:02 UTC     builder@monarth:/volume/build/junos/18.2/service/18.2X75-D61.10/obj/powerpc/junos/bsd/kernels/JUNIPER-PPC/kernel  powerpc


In [ ]:
# Get the output using RPC (XML)

Example:
    % vi python_fpc_info.py

    #!/usr/bin/env python

    from jnpr.junos import Device
    from lxml import etree

    with Device() as dev:

        fpcs_xpath=dev.rpc.get_fpc_information()
        print(etree.dump(fpcs_xpath))
        fpcs=fpcs_xpath.iter()
        for fpc in fpcs:
            if fpc.tag=='fpc':
                print(fpc.findtext('slot'), fpc.findtext('state'),fpc.findtext('temperature'),fpc.findtext('cpu-total'))
                
Output:
    
    ccl@ptx1k-msft> op python_fpc_info.py 
    <fpc-information style="brief">
    <fpc>
    <slot>0</slot>
    <state>Online</state>
    <temperature celsius="36">36</temperature>
    <cpu-total>24</cpu-total>
    <cpu-interrupt>0</cpu-interrupt>
    <cpu-1min-avg>23</cpu-1min-avg>
    <cpu-5min-avg>23</cpu-5min-avg>
    <cpu-15min-avg>23</cpu-15min-avg>
    <memory-dram-size>32768</memory-dram-size>
    <memory-heap-utilization>85</memory-heap-utilization>
    <memory-buffer-utilization>84</memory-buffer-utilization>
    </fpc>
    </fpc-information>
    None
    ('0', 'Online', '36', '24')
    

Example: Now # out print(etree.dump(fpcs_xpath))
    
Output: 
    
    ccl@ptx1k-msft> op python_fpc_info.py 
    ('0', 'Online', '36', '23')
    


In [ ]:
# Collect data from CLI and SHELL with python op script:
    
Example:
    
    
ccl@ptx1k-msft> file show /var/db/scripts/op/python_log_collector.py 
 
#!/usr/bin/env python
 
from lxml import etree
import jcs
import datetime
from jnpr.junos import Device
import subprocess
 
def cli_data(dev):
 
        with open('/var/tmp/cli-cmds.txt','r') as f1:
                cmds=f1.readlines()
                for cmd in cmds:
                        output=etree.tostring(dev.rpc.cli(command=cmd))
                        file_name = '/var/tmp/cli-data-collection/{}_{}'.format(cmd.strip(), datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')).replace(' ', '_')
                        write_to_file(cmd,output,file_name)
 
 
def shell_data(dev):
 
        with open('/var/tmp/shell-cmds.txt','r') as f1:
                cmds=f1.readlines()
                for cmd in cmds:
                        output=subprocess.check_output(cmd,shell=True)
                        file_name = '/var/tmp/shell-data-collection/{}_{}'.format(cmd.strip(), datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')).replace(' ', '_')
                        write_to_file(cmd,output,file_name)
 
 
def write_to_file(cmd,data,file_name):
 
        with open(file_name,'a') as f:
 
                f.write(data)
                f.write('\n\n')
 
 
if __name__ == '__main__':
 
        with Device() as dev:
 
                jcs.syslog('user.err','Start the data collection script!')
                dev.rpc.make_directory(directory='/var/tmp/cli-data-collection')
                cli_data(dev)
                dev.rpc.make_directory(directory='/var/tmp/shell-data-collection')
                shell_data(dev)

                
show log messages | last 20 
Jan 22 15:59:29  ptx1k-msft cscript.crypto: Start the data collection script!
Jan 22 15:59:29  ptx1k-msft mgd[71204]: UI_CHILD_EXITED: Child exited: PID 71207, status 1, command '/bin/mkdir'
Jan 22 15:59:30  ptx1k-msft mgd[71204]: UI_CHILD_EXITED: Child exited: PID 71366, status 1, command '/bin/mkdir'

ccl@ptx1k-msft> file show /var/tmp/cli-cmds.txt 
show system uptime
show version brief

ccl@ptx1k-msft> file show /var/tmp/shell-cmds.txt                                                                   
uname -a
cli -c "show chassis fpc"


Output: 
 
% ls -l | grep data
drwxr-xr-x  2 ccl            wheel         512 Jan 22 15:59 cli-data-collection
drwxr-xr-x  2 ccl            wheel         512 Jan 22 15:59 shell-data-collection
    
/var/tmp/cli-data-collection
% ls -l
total 48
-rw-r--r--  1 ccl  wheel   334 Jan 22 15:55 show_system_uptime_20210122_155508
-rw-r--r--  1 ccl  wheel  4448 Jan 22 15:59 show_version_brief_20210122_155930

% cat show_system_uptime_20210122_155508
<output>
Current time: 2021-01-22 15:55:08 UTC
Time Source:  LOCAL CLOCK 
System booted: 2020-12-31 20:31:19 UTC (3w0d 19:23 ago)
Protocols started: 2020-12-31 20:32:28 UTC (3w0d 19:22 ago)
Last configured: 2021-01-22 15:12:57 UTC (00:42:11 ago) by ccl
 3:55PM  up 21 days, 19:24, 2 users, load averages: 0.80, 0.83, 0.78
</output>


/var/tmp/shell-data-collection
% ls -l
total 16
-rw-r--r--  1 ccl  wheel  289 Jan 22 15:59 cli_-c_"show_chassis_fpc"_20210122_155930
-rw-r--r--  1 ccl  wheel  364 Jan 22 15:59 uname_-a_20210122_155930
    

% cat cli_-c_\"show_chassis_fpc\"_20210122_155930 
                     Temp  CPU Utilization (%)   CPU Utilization (%)  Memory    Utilization (%)
Slot State            (C)  Total  Interrupt      1min   5min   15min  DRAM (MB) Heap     Buffer
  0  Online            36     23          0       23     23     23    32768      84         84

In [ ]:
# PyEZ Event Scripts:

Event script requires Junos configuration to trigger the execution of script and it can take arguments as well

https://www.juniper.net/documentation/en_US/junos/topics/task/configuration/junos-script-automation-event-script-arguments-configuring-and-using.html

Example: This is a simple HOW-TO example to monitor interface status, collect some command output upon interface flapping, and then write the output into a file locally.

Junos Config: 

set event-options policy interface_status_check events snmp_trap_link_down
set event-options policy interface_status_check attributes-match snmp_trap_link_down.interface-name matches et-
set event-options policy interface_status_check then event-script python_link_flap.py arguments interface "{$$.interface-name}"
set event-options event-script optional
set event-options event-script file python_link_flap.py python-script-user ccl

Script: 
    
ccl@ptx1k-msft> file show /var/db/scripts/event/python_link_flap.py 

#!/usr/bin/env python

import argparse

from jnpr.junos import Device 
import jcs 
import datetime 
from lxml import etree 
import sys 
import platform

if __name__ =='__main__': 
    
    parser = argparse.ArgumentParser() 
    parser.add_argument('-interface') 
    args = parser.parse_args() 
    jcs.syslog('user.err','Python version {}'.format(platform.sys.version)) 
    int_name=args.interface 
    
    with Device() as dev: 
        
        jcs.syslog('user.err','{} goes down, start the data collection!!!!!'.format(int_name))

        with open('/var/tmp/cmds.txt','r') as cmd_file:
            cmds=cmd_file.readlines()
            for cmd in cmds:
                jcs.syslog('user.err','Collecting {}\n'.format(cmd))
                output=etree.tostring(dev.rpc.cli(command=cmd))

                file_name='/var/tmp/{}_{}'.format(cmd.strip(),datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')).replace(' ','_')
                with open(file_name,'a') as f:
                    f.write(output)
                    f.write('\n\n')
                    
/var/tmp
% cat cmds.txt
show chassis fpc
show system core-dumps
                    
Log Message:   
    
    Jan 22 21:21:53  ptx1k-msft mib2d[14373]: SNMP_TRAP_LINK_DOWN: ifIndex 530, ifAdminStatus down(2), ifOperStatus down(2), ifName et-0/0/0:1
    Jan 22 21:21:53  ptx1k-msft fpc0 T6E(0/0) link[0 1] Down: phy_link_ok: 1 mac_local_fault:0 mac_remote_fault:1 asic_fault: 0
    Jan 22 21:21:53  ptx1k-msft cscript.crypto: Python version 2.7.8 (default, Dec  4 2020, 22:47:55)  [GCC 4.2.1 Compatible Juniper Clang 3.7.1 (git@psd-tools-git01.juniper.net:tool
    Jan 22 21:21:54  ptx1k-msft cscript.crypto: et-0/0/0:1 goes down, start the data collection!!!!!
    Jan 22 21:21:54  ptx1k-msft cscript.crypto: Collecting show chassis fpc
    Jan 22 21:21:54  ptx1k-msft cscript.crypto: Collecting show system core-dumps

Output:
                                                                                                                                                  
/var/tmp                                                                                                                                                 
-rw-r--r--  1 ccl            wheel         308 Jan 22 21:21 show_chassis_fpc_20210122_212154
-rw-r--r--  1 ccl            wheel          66 Jan 22 21:21 show_system_core-dumps_20210122_212154      
                                                                                                                                                  
                                                                                                                                               
% cat  show_chassis_fpc_20210122_212154
<output>
                     Temp  CPU Utilization (%)   CPU Utilization (%)  Memory    Utilization (%)
Slot State            (C)  Total  Interrupt      1min   5min   15min  DRAM (MB) Heap     Buffer
  0  Online            36     24          0       23     23     23    32768      84         84
</output>

                                                                                                                                                  
% cat show_system_core-dumps_20210122_212154
<output>
/var/crash/*core*: No such file or directory
</output>
                                                                                                                                                  
NOTES:

To determine whether the script is triggered successfully, please check the escript.log file under /var/log

ccl@ptx1k-msft> show log escript.log                                                                                                                                                  
Jan 22 21:21:53 event script processing begins
Jan 22 21:21:53 running event script 'python_link_flap.py'
Jan 22 21:21:53 opening event script '/var/db/scripts/event/python_link_flap.py'
Jan 22 21:21:53 reading event script 'python_link_flap.py'
Jan 22 21:21:54 event script execution successful for 'python_link_flap.py' with return: 0
Jan 22 21:21:54 finished event script 'python_link_flap.py'
Jan 22 21:21:54 event script processing ends                                                                                                                                                 
                                                                                                                                                  